In [ ]:
import argparse
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import os
from torchvision import datasets, models
from collections import OrderedDict
import sys

sys.argv = [
    "train.py",
    "/kaggle/input/oxford-102-flower-dataset/102 flower/flowers",
    "--save_dir", "/kaggle/working/",
    "--arch", "vgg16",
    "--learning_rate", "0.001",
    "--epochs", "10",
    "--gpu"
]

def get_input_args():
    parser = argparse.ArgumentParser(description="Train a deep learning model on a dataset")
    
    parser.add_argument('data_dir', type=str, help='Path to dataset directory')
    parser.add_argument('--save_dir', type=str, default='./', help='Directory to save checkpoint')
    parser.add_argument('--arch', type=str, default='mobilenet_v2', choices=['mobilenet_v2', 'vgg16', 'vgg13'], help='Model architecture')
    parser.add_argument('--learning_rate', type=float, default=0.001, help='Learning rate for training')
    parser.add_argument('--hidden_units', type=int, default=512, help='Number of hidden units in classifier')
    parser.add_argument('--epochs', type=int, default=10, help='Number of epochs for training')
    parser.add_argument('--gpu', action='store_true', help='Use GPU for training')

    return parser.parse_args()

def load_data(data_dir):
    train_dir = os.path.join(data_dir, 'train')
    valid_dir = os.path.join(data_dir, 'valid')
    
    # Define transforms
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    valid_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load datasets
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
    valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)

    return train_loader, valid_loader, train_dataset.class_to_idx

def build_model(arch, hidden_units):
    if arch == "mobilenet_v2":
        model = models.mobilenet_v2(pretrained=True)
        input_size = model.classifier[1].in_features
    elif arch == "vgg16":
        model = models.vgg16(pretrained=True)
        input_size = model.classifier[0].in_features
    elif arch == "vgg13":
        model = models.vgg13(pretrained=True)
        input_size = model.classifier[0].in_features
    else:
        raise ValueError(f"Unsupported architecture: {arch}")

    # Freeze pretrained parameters
    for param in model.parameters():
        param.requires_grad = False

    # Define a new classifier
    classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(input_size, hidden_units)),
        ('relu', nn.ReLU()),
        ('dropout', nn.Dropout(0.2)),
        ('fc2', nn.Linear(hidden_units, 102)),  # 102 flower categories
        ('output', nn.LogSoftmax(dim=1))
    ]))
    if arch == "mobilenet_v2":
        model.fc = classifier
    else:
        model.classifier = classifier

    return model
def train_model(model, train_loader, valid_loader, device, epochs, learning_rate):
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

    model.to(device)

    for epoch in range(epochs):
        train_loss = 0
        model.train()
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation step
        model.eval()
        valid_loss = 0
        accuracy = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                loss = criterion(output, labels)
                valid_loss += loss.item()

                # Calculate accuracy
                probab = torch.softmax(output, dim=1) 
                top_p, top_class = probab.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        print(f"Epoch {epoch+1}/{epochs}.. "
              f"Train Loss: {train_loss/len(train_loader):.3f}.. "
              f"Validation Loss: {valid_loss/len(valid_loader):.3f}.. "
              f"Validation Accuracy: {accuracy/len(valid_loader)*100:.2f}%")

    return model, optimizer
def save_checkpoint(model, save_dir, arch, class_to_idx):
    checkpoint = {
        'arch': arch,
        'state_dict': model.state_dict(),
        'classifier': model.classifier,
        'class_to_idx': class_to_idx
    }
    save_path = os.path.join(save_dir, 'checkpoint.pth')
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved at: {save_path}")

def main():
    args = get_input_args()
    device = torch.device("cuda" if args.gpu and torch.cuda.is_available() else "cpu")

    train_loader, valid_loader, class_to_idx = load_data(args.data_dir)
    model = build_model(args.arch, args.hidden_units)

    model, optimizer = train_model(model, train_loader, valid_loader, device, args.epochs, args.learning_rate)
    save_checkpoint(model, args.save_dir, args.arch, class_to_idx)

if __name__ == '__main__':
    main()


In [ ]:
import os
print(os.listdir("/kaggle/working/"))
##torch.save(models.state_dict(), "/kaggle/working/checkpoint.pth")
print(sys.argv)



In [ ]:
import argparse
import torch
import torchvision.transforms as transforms
from PIL import Image
import json
from torchvision import models
import os
import sys
sys.argv = [
    "predict.py",
    "/kaggle/input/oxford-102-flower-dataset/102 flower/flowers/train/7/image_08103.jpg",
    "/kaggle/working/checkpoint.pth", 
    "--name_file", "/kaggle/input/cat-to-name-json/cat_to_name.json", 
    "--top_k", "5",
    "--gpu"
]

def get_input_args():
    parser = argparse.ArgumentParser(description="Predict the class of an image using a trained deep learning model")
    
    parser.add_argument('image_path', type=str, help='Path to input image')
    parser.add_argument('checkpoint', type=str, help='Path to model checkpoint')
    parser.add_argument('--name_file', type=str, help='Path to JSON file mapping categories to names')
    parser.add_argument('--top_k', type=int, default=1, help='Return top K most likely classes')
    parser.add_argument('--gpu', action='store_true', help='Use GPU for inference')

    return parser.parse_args()

def load_checkpoint(filepath):
    
    checkpoint = torch.load(filepath)
    model = models.vgg16(pretrained=True)
    model.name ="vgg16"

    for param in model.parameters():
        param.requires_grad =False
        
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    return model

def process_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert("RGB") 
    
    image = transform(image)
    
    numpy_image = np.array(image)
     
    return torch.unsqueeze(image, 0)
def predict(image, model, cat_to_name=None, topk=5, device="cpu"):
    model.eval()
    image = image.to(device)
    
    with torch.no_grad():
        output = model.forward(image)
    
    probab = torch.exp(output)
    probs, indices = probab.topk(topk, dim=1)
    
    list_probs = probs.tolist()[0]
    list_indices = indices.tolist()[0]
    
    index_map = {v: k for k, v in model.class_to_idx.items()}
    classes = [index_map[idx] for idx in list_indices]

    # Convert class indices to actual names if `cat_to_name` is provided
    if cat_to_name:
        labels = [cat_to_name[c] for c in classes]
        return list_probs, labels
    
    return list_probs, classes
    
def print_predictions(probabilities, classes, cat_to_name=None):
    labels = [cat_to_name.get(c, f"Class {c}") for c in classes] if cat_to_name else classes

    for i, (probab, label, c) in enumerate(zip(probabilities, labels, classes), 1):
        print(f"{i}) {probab*100:.2f}% - {label.title()} (Class {c})")

def main():
    args = get_input_args()

    with open(args.name_file, 'r') as f:
                cat_to_name = json.load(f)

    model = load_checkpoint(args.checkpoint)
    
    image = process_image(args.image_path)
    
    device = torch.device("cuda" if args.gpu and torch.cuda.is_available() else "cpu")
    
    model.to(device)

    probs, classes = predict(image, model, cat_to_name, args.top_k, device)
    print_predictions(probs, classes)
    
      
if __name__ == '__main__':
    main()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_path = "/kaggle/input/oxford-102-flower-dataset/102 flower/flowers/train/7/image_08103.jpg"

# Load and display the image
img = mpimg.imread(image_path)
plt.imshow(img)
plt.axis("off")  # Hide axis
plt.show()


In [ ]:
import os

dataset_path = "/kaggle/input/oxford-102-flower-dataset/102 flower/flowers/train"
class_folders = os.listdir(dataset_path)

for class_folder in class_folders[:5]:  # Checking first 5 class folders
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        print(f"Checking {class_path}:")
        print(os.listdir(class_path)[:10])  # Print first 10 files inside
